<h1>
Model for Business Site Recommendation
</h1>

<ul>
<li>Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page</li>
<li>Get the geographical coordinates of the neighborhoods</li>
<li>Obtain the venue data for the neighborhoods from Foursquare API</li>
<li>Explore and cluster the neighborhoods</li>
<li>Select the best cluster to open a new shopping mall</li>
</ul>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


In [2]:
#Scrap data from Wikipedia page into a DataFrame
data = requests.get("https://en.m.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore").text

In [3]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighborhoodList = []
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("a"):
    neighborhoodList.append(row.text)

In [5]:
b_df = pd.DataFrame({"Neighborhood": neighborhoodList[15:]})
b_df['Neighborhood'].replace('', inplace=True)
b_df.head(10)


,Neighborhood
0,Cantonment
1,British Raj
2,Richmond Town
3,Austin Town
4,Murphy Town
5,Fraser Town
6,Cox Town
7,Domlur
8,
9,HAL


In [6]:
b_df.shape


(187, 1)

In [7]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Banaglore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:

# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in b_df["Neighborhood"].tolist() ]

KeyboardInterrupt: 

In [ ]:
coords


[[12.976600000000076, 77.59930000000008],
 [12.966180000000065, 77.58690000000007],
 [12.962330000000065, 77.60123000000004],
 [12.963480000000061, 77.61297000000008],
 [12.980030000000056, 77.62776000000008],
 [12.998940000000061, 77.61276000000004],
 [12.995940000000076, 77.62644000000006],
 [12.943290000000047, 77.65602000000007],
 [12.966180000000065, 77.58690000000007],
 [12.960930000000076, 77.66419000000008],
 [13.030060000000049, 77.49526000000003],
 [12.966180000000065, 77.58690000000007],
 [12.908585406663308, 77.57026481360015],
 [13.005440000000021, 77.55693000000008],
 [13.00632005596653, 77.56839983128529],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.975498768188668, 77.52385771447123],
 [12.970140000000072, 77.57589000000007],
 [12.964216295261721, 77.57681755546776],
 [12.969388412598315, 77.57950705966006],
 [13.014830000000075, 77.57771000000008],
 [12.961200000000076, 77.5107100000

In [ ]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [ ]:
b_df['Latitude'] = df_coords['Latitude']
b_df['Longitude'] = df_coords['Longitude']


In [ ]:
print(b_df.shape)
b_df

(187, 3)


,Neighborhood,Latitude,Longitude
0,Cantonment,12.976600,77.599300
1,British Raj,12.966180,77.586900
2,Richmond Town,12.962330,77.601230
3,Austin Town,12.963480,77.612970
4,Murphy Town,12.980030,77.627760
5,Fraser Town,12.998940,77.612760
6,Cox Town,12.995940,77.626440
7,Domlur,12.943290,77.656020
8,,12.966180,77.586900
9,HAL,12.960930,77.664190


In [ ]:
b_df.to_csv("b_df.csv", index=False)

Create a map of Bangalore with neighborhoods superimposed on top

In [ ]:
# get the coordinates of Bangalore
address = 'Bangalore,India'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9767936, 77.590082.


In [ ]:

map_b = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, neighborhood in zip(b_df['Latitude'], b_df['Longitude'], b_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_b)  
    
map_b

In [ ]:
map_b.save('map_b.html')

In [ ]:
CLIENT_ID = 'NZ0DM2ODTFWSE31NHMSMVZCX4A5YZMF4WKKVGGZTNCXVRXEM' # your Foursquare ID
CLIENT_SECRET = 'FDAGFHSOHZ2YTIACFGF2XY0ULG5A0EBLJTJETK3OVSCEOYJV' # your Foursquare Secret
VERSION = '20210609' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NZ0DM2ODTFWSE31NHMSMVZCX4A5YZMF4WKKVGGZTNCXVRXEM
CLIENT_SECRET:FDAGFHSOHZ2YTIACFGF2XY0ULG5A0EBLJTJETK3OVSCEOYJV


In [ ]:

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(b_df['Latitude'], b_df['Longitude'],b_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(12513, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cantonment,12.9766,77.5993,Hard Rock Cafe Bengaluru,12.976179,77.601543,American Restaurant
1,Cantonment,12.9766,77.5993,M. Chinnaswamy Stadium,12.978144,77.599223,Cricket Ground
2,Cantonment,12.9766,77.5993,Cubbon Park,12.977042,77.595277,Park
3,Cantonment,12.9766,77.5993,Corner House,12.972979,77.599971,Ice Cream Shop
4,Cantonment,12.9766,77.5993,Church Street Social,12.975559,77.602579,Pub


In [ ]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
100 Feet Road,100,100,100,100,100,100
Anekal,4,4,4,4,4,4
Anjanapura,6,6,6,6,6,6
Anugraha,27,27,27,27,27,27
Arekere,71,71,71,71,71,71
Attibele,8,8,8,8,8,8
Austin Town,100,100,100,100,100,100
Avenue Road,100,100,100,100,100,100
BTM Layout,92,92,92,92,92,92


In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


In [ ]:
venues_df['VenueCategory'].unique()[:236]


array(['American Restaurant', 'Cricket Ground', 'Park', 'Ice Cream Shop',
       'Pub', 'Chinese Restaurant', 'Toy / Game Store', 'Burger Joint',
       'Plaza', 'Hotel', 'Deli / Bodega', 'Lounge', 'Breakfast Spot',
       'Shopping Mall', 'Italian Restaurant', 'Furniture / Home Store',
       'Asian Restaurant', 'Japanese Restaurant', 'Bookstore',
       'French Restaurant', 'Bubble Tea Shop', 'Brewery', 'Steakhouse',
       'Cupcake Shop', 'Afghan Restaurant', 'Indian Restaurant',
       'Sushi Restaurant', 'Clothing Store', 'Café', 'Bakery',
       'Cocktail Bar', 'Pizza Place', 'Coffee Shop', 'Tea Room',
       'Andhra Restaurant', 'Athletics & Sports', 'Gym / Fitness Center',
       'Arcade', 'Electronics Store', 'Fried Chicken Joint',
       'Vegetarian / Vegan Restaurant', 'Bed & Breakfast',
       'Soccer Stadium', "Women's Store", 'Gym', 'Korean Restaurant',
       'Dessert Shop', 'Wine Bar', 'Mediterranean Restaurant',
       'Arts & Crafts Store', 'Seafood Restaurant', 'Race

In [ ]:
"Neighborhood" in venues_df['VenueCategory'].unique()


True

In [ ]:
b_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
b_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [b_onehot.columns[-1]] + list(b_onehot.columns[:-1])
b_onehot = b_onehot[fixed_columns]

print(b_onehot.shape)
b_onehot.head()

(12513, 238)


,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racecourse,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Cantonment,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
b_grouped = b_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(b_grouped.shape)
b_grouped

(127, 238)


,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racecourse,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,100 Feet Road,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000,0.00,0.000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00

In [ ]:
b_grouped.to_csv("b_grp.csv", index=False)

In [ ]:
len(b_grouped[b_grouped["Restaurant"] > 0])


62

In [ ]:

b_mall = b_grouped[["Neighborhoods","Restaurant"]]
b_mall


,Neighborhoods,Restaurant
0,100 Feet Road,0.010000
1,Anekal,0.000000
2,Anjanapura,0.000000
3,Anugraha,0.074074
4,Arekere,0.014085
5,Attibele,0.000000
6,Austin Town,0.000000
7,Avenue Road,0.020000
8,BTM Layout,0.010870
9,Bagmane Tech Park,0.000000


Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Bangalore into 3 clusters.

In [ ]:
# set number of clusters
from sklearn.metrics import accuracy_score
bclusters = 3


b_clustering = b_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=bclusters, random_state=0).fit(b_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


C:\Users\nk\AppData\Local\Temp\ipykernel_1924\2378773592.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  b_clustering = b_mall.drop(["Neighborhoods"], 1)


array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
bmerged = b_mall.copy()

# add clustering labels
bmerged["Cluster Labels"] = kmeans.labels_

In [ ]:

bmerged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bmerged.head()

,Neighborhood,Restaurant,Cluster Labels
0,100 Feet Road,0.010000,0
1,Anekal,0.000000,0
2,Anjanapura,0.000000,0
3,Anugraha,0.074074,1
4,Arekere,0.014085,0


In [ ]:
bmerged = bmerged.join(b_df.set_index("Neighborhood"), on="Neighborhood")

print(bmerged.shape)
bmerged.head() # check the last columns!

(187, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,100 Feet Road,0.010000,0,12.908585,77.570265
1,Anekal,0.000000,0,12.709560,77.698040
2,Anjanapura,0.000000,0,12.858110,77.559100
3,Anugraha,0.074074,1,12.961200,77.510710
4,Arekere,0.014085,0,12.885670,77.596730


In [ ]:
print(bmerged.shape)
bmerged.sort_values(["Cluster Labels"], inplace=True)
bmerged


(187, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,100 Feet Road,0.010000,0,12.908585,77.570265
103,SH39,0.000000,0,13.130685,77.484375
103,SH39,0.000000,0,13.130685,77.484375
104,SH87,0.000000,0,12.976360,77.601780
105,Sadashivanagar,0.000000,0,13.014830,77.577710
107,Seshadripuram,0.020000,0,12.993550,77.579880
108,Shivajinagar,0.000000,0,12.987180,77.603980
109,Thavarekere,0.010000,0,12.922400,77.610270
110,Ulsoor,0.000000,0,12.989080,77.627950
102,SH35,0.000000,0,12.797484,77.780368


In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(bclusters)
ys = [i+x+(i*x)**2 for i in range(bclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bmerged['Latitude'], bmerged['Longitude'], bmerged['Neighborhood'], bmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
map_clusters.save('map_clusters.html')

In [ ]:

bmerged.loc[bmerged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,100 Feet Road,0.010000,0,12.908585,77.570265
103,SH39,0.000000,0,13.130685,77.484375
103,SH39,0.000000,0,13.130685,77.484375
104,SH87,0.000000,0,12.976360,77.601780
105,Sadashivanagar,0.000000,0,13.014830,77.577710
107,Seshadripuram,0.020000,0,12.993550,77.579880
108,Shivajinagar,0.000000,0,12.987180,77.603980
109,Thavarekere,0.010000,0,12.922400,77.610270
110,Ulsoor,0.000000,0,12.989080,77.627950
102,SH35,0.000000,0,12.797484,77.780368


In [ ]:
bmerged.loc[bmerged['Cluster Labels'] == 1]


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
3,Anugraha,0.074074,1,12.961200,77.51071
78,Mysore Road metro station,0.038462,1,12.946340,77.53057
84,Nagarbhavi,0.052632,1,12.956240,77.50936
77,Mysore Road Satellite Bus Station,0.050000,1,12.953810,77.54333
32,Domlur,0.063830,1,12.943290,77.65602
121,Yeshwanthpur,0.076923,1,13.029540,77.54020
118,Whitefield,0.060606,1,12.979320,77.73370
117,Vijayanagara Empire,0.090909,1,13.076000,77.65240
116,Vijayanagar,0.090909,1,13.076000,77.65240
115,Vidyaranyapura,0.071429,1,13.079290,77.55912


In [ ]:
bmerged.loc[bmerged['Cluster Labels'] == 2]


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
83,NICE Road,0.200000,2,12.893151,77.479381
29,Chikkabanavara,0.200000,2,13.072135,77.513750
17,Basavanna,0.166667,2,13.070794,77.539413
106,Sarjapura,0.200000,2,12.860720,77.783600
112,Uttarahalli,0.142857,2,12.897570,77.528300
